In [1]:
import sys;
sys.path.insert(0, '..')

## Chapter 10 Code Snippets and Listings

### Encoding a simple function (10.1.1)

Let's learn how to encode key-value pairs where the value is the sum of the binary digits in the key (for example, the sum of the binary digits in the key 011 is 2). If we use a key register with $n = 3$ qubits, the sum of the binary digits of an integer $k$, with $0 \le k < 2^n$, is given by the expression

$$
s(k) = k_2 + k_1 + k_0
$$

where $k_0$, $k_1$, and $k_2$ are 0 or 1 (the binary digits of a key).

First, we create a circuit with two registers:

In [2]:
from sim_circuit import QuantumRegister, QuantumCircuit

n_key = 3
n_value = 2

key = QuantumRegister(n_key)
value = QuantumRegister(n_value)
qc = QuantumCircuit(key, value)

We start with an equal superposition in both registers:

In [3]:
for i in range(n_key): # put the key register in equal superposition
    qc.h(key[i])

for i in range(n_value): # put the value register in equal superposition
    qc.h(value[i])

**Binary decomposition of integers**

An integer $k$ with $0 \le k < 2^n$ can be represented as a binary string with $n$ digits.
We can generate the positions  of the digits that are 1 using the following code:

In [4]:
def one_digits(n, k):
    for i in range(n):
        if k & (1 << i):
            yield i

For example, the binary form of $k = 4$ with $n = 3$ digits is 100, so the function `one_digits` will yield position 2:

In [5]:
n = 3
k = 4

for i in one_digits(n, k):
    print(i)

2


The decimal form of an integer $k$, where $0 \le k < 2^n$, is the sum of powers of 2 corresponding to these positions.
The value $k = 4$ is decomposed into powers of 2 as follows:

In [6]:
sum([2**i for i in one_digits(n,k)])

4

We can use this sum representation of $k$ to obtain a sum representation of $k\theta$, for a given angle $\theta$.
For example, if $n = 3$ and $\theta = \frac{\pi}{4}$, the multiple $4\theta$ can be expressed as:

In [7]:
sum([2**i for i in one_digits(n,k)])

4

The following code snippet verifies that the decomposition is correct:

In [8]:
from math import pi
from util import is_close

theta = pi/4
assert is_close(4*theta, sum([2**i * theta for i in one_digits(n, k)]))

We use a sequence of controlled phase rotations to create geometric quantum states:

In [9]:
for j in range(n_key):
    for i in range(n_value):
        qc.cp(pi / 2 ** (n_key-1-i), key[j], value[i])

In [10]:
state = qc.run()

In [11]:
from util import print_state_table

print_state_table(state)


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        00000   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
1        00001   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
2        00010   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
3        00011   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
4        00100   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
5        00101   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
6        00110   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
7        00111   0.1768 + i0.0000    0.1768        0.00°   ████                      0.0312
8        01000   0.1768 + i0.0000    0.1768        0.00°   ████      

In [12]:
n_key = 3
n_value = 2

key = QuantumRegister(n_key)
value = QuantumRegister(n_value)
qc = QuantumCircuit(key, value)

for i in range(n_key):
    qc.h(key[i])

for i in range(n_value):
    qc.h(value[i])
    
for j in range(n_key):
    for i in range(n_value):
        qc.cp(pi / 2 ** (n_key-1-i), key[j], value[i])

Finally, we apply the IQFT to the value register:

In [13]:
qc.iqft(value, swap=True)

Remember from chapter 8, when we prepare a geometric sequence with qubits in reverse order, we can skip the swaps in the IQFT.
We will apply controlled phase rotations with the same angles as above to the qubits in reverse order (qubit $i$ becomes qubit $n - i -1$) using the code below:

In [14]:
n_key = 3
n_value = 2

key = QuantumRegister(n_key)
value = QuantumRegister(n_value)
qc = QuantumCircuit(key, value)

for i in range(n_key):
    qc.h(key[i])

for i in range(n_value):
    qc.h(value[i])
    
for j in range(n_key):
    for i in range(n_value):
        qc.cp(pi / 2**i, key[j], value[i])

qc.iqft(value[::-1], swap=False)

In [15]:
from util import grid_state

state = qc.run()
grid_state(state, n_key, neg=False, show_probs=False)



╒════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│        │ 0 = 000   │ 1 = 001   │ 2 = 010   │ 3 = 011   │ 4 = 100   │ 5 = 101   │ 6 = 110   │ 7 = 111   │
╞════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 3 = 11 │           │           │           │           │           │           │           │           │
├────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 2 = 10 │           │           │           │           │           │           │           │           │
├────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 1 = 01 │           │           │           │           │           │           │           │           │
├────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 0 = 00 │           │           │ 

### Encoding the knapsack problem (10.1.2)

Let's encode the value for each possible selection.
First, we create a circuit with just the selection and value registers, and we put both in equal superposition:

In [16]:
n_key = 3
n_value = 3

key = QuantumRegister(n_key)
value = QuantumRegister(n_value)
qc = QuantumCircuit(key, value)

for i in range(n_key):
    qc.h(key[i])

for i in range(n_value):
    qc.h(value[i])

We can represent the value function with the following list of tuples:

In [17]:
terms = [(2, [0]), (3, [1]), (1, [2])]

We can use the following for-loop to apply the controlled phase rotations:

In [18]:
for (coeff, vars) in terms:
    for i in range(n_value):
        qc.cp(pi * 2 ** -i * coeff, key[vars[0]], value[i])

Next, we apply the IQFT to the value register:

In [19]:
qc.iqft(value[::-1], swap=False)

In [20]:
grid_state(qc.run(), n_key, False, False)



╒═════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│         │ 0 = 000   │ 1 = 001   │ 2 = 010   │ 3 = 011   │ 4 = 100   │ 5 = 101   │ 6 = 110   │ 7 = 111   │
╞═════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7 = 111 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 6 = 110 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 5 = 101 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 4 = 100 │           │   

We can represent the weight function with the following list of tuples:

In [21]:
terms = [(3, [0]), (2, [1]), (1, [2])]

We can encode the weight function using the code below:

In [22]:
n_key = 3
n_weight = 3

key = QuantumRegister(n_key)
weight = QuantumRegister(n_weight)
qc = QuantumCircuit(key, weight)

for i in range(n_key):
    qc.h(key[i])

for i in range(n_weight):
    qc.h(weight[i])
    
for (coeff, vars) in terms:
    for i in range(n_weight):
        qc.cp(pi * 2 ** -i * coeff, key[vars[0]], weight[i])

qc.iqft(weight[::-1], swap=False)      

In [23]:
grid_state(qc.run(), n_key, False, False)



╒═════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│         │ 0 = 000   │ 1 = 001   │ 2 = 010   │ 3 = 011   │ 4 = 100   │ 5 = 101   │ 6 = 110   │ 7 = 111   │
╞═════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7 = 111 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 6 = 110 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 5 = 101 │           │           │           │           │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 4 = 100 │           │   

### Encoding polynomials of binary variables (10.1.3)

The list of tuples used to encode the function $f(k) = k^2 + 2$ for $n = 2$:

In [24]:
terms = [(4, [1]), (4, [1, 0]), (1, [0]), (2, [])]

Listing 10.1 Function to create the circuit that encodes a given term

In [25]:
def encode_term(coeff, vars, circuit, key, value):
    for i in range(len(value)):
        if len(vars) > 1:
            circuit.mcp(pi * 2 ** -i * coeff, [key[j] for j in vars], value[i])
        elif len(vars) > 0:
            circuit.cp(pi * 2 ** -i * coeff, key[vars[0]], value[i])
        else:
            circuit.p(pi * 2 ** -i * coeff, value[i])

Listing 10.2 Create a circuit for encoding a polynomial in a quantum state

In [26]:
def build_polynomial_circuit(key_size, value_size, terms):
    key = QuantumRegister(key_size)
    value = QuantumRegister(value_size)
    circuit = QuantumCircuit(key, value)

    for i in range(len(key)):
        circuit.h(key[i])

    for i in range(len(value)):
        circuit.h(value[i])

    for (coeff, vars) in terms:
        encode_term(coeff, vars, circuit, key, value)

    circuit.iqft(value[::-1], swap=False)

    return circuit

Now we can encode our polynomial of binary variables. We will need $m = 4$ value qubits to encode the outputs.

In [27]:
n_key = 2
n_value = 4
    
qc = build_polynomial_circuit(n_key, n_value, terms)

In [28]:
grid_state(qc.run(), n_key, False, False)



╒═══════════╤══════════╤══════════╤══════════╤══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═══════════╪══════════╪══════════╪══════════╪══════════╡
│ 15 = 1111 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 14 = 1110 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 13 = 1101 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 12 = 1100 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 11 = 1011 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 10 = 1010 │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 9 = 1001  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 8 = 1000  

Let's look at another example.
This time, let's consider the function $f(k) = k^2 - 5k + 7$ with integer inputs $\{0, 1, 2, 3 \}$ (we need $n = 2$ qubits to represend inputs).

In [29]:
terms = [(-6, [1]), (4, [1, 0]), (-4, [0]), (7, [])]

In [30]:
n_key = 2
n_value = 3
    
qc = build_polynomial_circuit(n_key, n_value, terms)

In [31]:
grid_state(qc.run(), n_key, False, False)



╒═════════╤══════════╤══════════╤══════════╤══════════╕
│         │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═════════╪══════════╪══════════╪══════════╪══════════╡
│ 7 = 111 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 6 = 110 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 5 = 101 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 4 = 100 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 3 = 011 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 2 = 010 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 1 = 001 │          │          │          │          │
├─────────┼──────────┼──────────┼──────────┼──────────┤
│ 0 = 000 │          │          │          │  

### Complexity of polynomial encoding circuits (10.1.4)

For the example circuit above, the total number of gates is:

In [32]:
n = 2
m = 3
terms = [(-6, [1]), (4, [1, 0]), (-4, [0]), (7, [])]

In [33]:
qc = build_polynomial_circuit(n, m, terms)
t = len(terms)
assert len(qc.transformations) == n + (t + 2)*m + m*(m - 1 )/2

### Representing negative values (10.1.5)

To encode the function $f(k) = k^2 - 3$ for integer inputs $\{0, 1, 2, 3 \}$, we express the function as a polynomial of binary variables:

In [34]:
terms = [(4, [1]), (4, [1, 0]), (1, [0]), (-3, [])]

In [35]:
n_key = 2
n_value = 4
    
qc = build_polynomial_circuit(n_key, n_value, terms)

In [36]:
grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒═══════════╤══════════╤══════════╤══════════╤══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═══════════╪══════════╪══════════╪══════════╪══════════╡
│ 7 = 0111  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 6 = 0110  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 5 = 0101  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 4 = 0100  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 3 = 0011  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 2 = 0010  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 1 = 0001  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0 = 0000  

### Searching for function values (10.2)

The two functions defined below create oracles that match 1 or 0 in the position `tag_bit`:

In [37]:
def oracle_match_1(bits, tag_bit):
    q = QuantumRegister(bits)
    qc = QuantumCircuit(q)

    qc.p(pi, tag_bit)

    return qc


def oracle_match_0(bits, tag_bit):
    q = QuantumRegister(bits)
    qc = QuantumCircuit(q)

    qc.x(q[tag_bit])
    qc.p(pi, tag_bit)
    qc.x(q[tag_bit])

    return qc

First, let's encode the function $f(k) = k + 1$ with $n = 2$ key qubits:

In [38]:
n_key = 2
n_value = 3

terms = [(2, [1]), (1, [0]), (1, [])]

prepare = build_polynomial_circuit(n_key, n_value, terms)

In [39]:
grid_state(prepare.run(), n_key, False, True)



╒═════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│         │ 0 = 00    │ 1 = 01    │ 2 = 10    │ 3 = 11    │
╞═════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7 = 111 │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 6 = 110 │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 5 = 101 │           │           │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 4 = 100 │           │           │           │      0.25 │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 3 = 011 │           │           │      0.25 │           │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 2 = 010 │           │      0.25 │           │           │
├─────────┼───────────┼───────────┼───────────┼───────────┤
│ 1 = 001 │      0.25 │           │           │           │
├─────────┼───────────┼───────────┼───

Now, we can create an oracle that matches 1 in the first digit of the value register and use it to create a Grover operator using the function `grover_circuit` from chapter 9:

In [40]:
from algo import grover_circuit

prepare = build_polynomial_circuit(n_key, n_value, terms)
oracle = oracle_match_1(n_key + n_value, n_key + n_value - 1)

qc = grover_circuit(prepare, oracle, 1)

In [41]:
grid_state(qc.run(), n_key, False, True)



╒═════════╤══════════╤══════════╤══════════╤═══════════════╕
│         │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11        │
╞═════════╪══════════╪══════════╪══════════╪═══════════════╡
│ 7 = 111 │          │          │          │               │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 6 = 110 │          │          │          │               │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 5 = 101 │          │          │          │               │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 4 = 100 │          │          │          │           1.0 │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 3 = 011 │          │          │          │               │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 2 = 010 │          │          │          │               │
├─────────┼──────────┼──────────┼──────────┼───────────────┤
│ 1 = 001 │          │          │          │               │
├─────────┼──────────┼

Let's look at searching for negative values.
For example, let's encode the function $f(k) = k^2 - 5$ for $n = 2$, and search for outputs less than -4.

In [42]:
n_key = 2
n_value = 4

terms = [(4, [1]), (4, [1, 0]), (1, [0]), (-5, [])]

prepare = build_polynomial_circuit(n_key, n_value, terms)

We will need to define an oracle that matches 1 in the first digit and 0 in the second digit of the value register.
We will use this oracle in our Grover iterate:

In [43]:
q = QuantumRegister(n_key + n_value)
oracle = QuantumCircuit(q)

oracle.x(q[n_key + n_value - 2]) 
oracle.cp(pi, n_key + n_value - 2, n_key + n_value - 1)
oracle.x(q[n_key + n_value - 2]) 

qc = grover_circuit(prepare, oracle, 1)

In [44]:
grid_state(qc.run(), n_key, True, True)



╒═══════════╤═══════════════╤══════════╤══════════╤══════════╕
│           │ 0 = 00        │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═══════════╪═══════════════╪══════════╪══════════╪══════════╡
│ 7 = 0111  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 6 = 0110  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 5 = 0101  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 4 = 0100  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 3 = 0011  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 2 = 0010  │               │          │          │          │
├───────────┼───────────────┼──────────┼──────────┼──────────┤
│ 1 = 0001  │               │          │          │  

### Finding zeros of polynomial functions (10.3)

Let's consider again a quadratic polynomial:

$$
p(k) = k^2 - 4
$$

for $0 \le k< 8$. We can encode this function in a quantum state with the following code:

In [45]:
n_key = 2
n_value = 4

terms = [(4, [1]), (4, [1, 0]), (1, [0]), (-4, [])]

qc = build_polynomial_circuit(n_key, n_value, terms)

In [46]:
grid_state(qc.run(), n_key, True, True)



╒═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│           │ 0 = 00    │ 1 = 01    │ 2 = 10    │ 3 = 11    │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 7 = 0111  │           │           │           │           │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 6 = 0110  │           │           │           │           │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 5 = 0101  │           │           │           │      0.25 │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 4 = 0100  │           │           │           │           │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 3 = 0011  │           │           │           │           │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 2 = 0010  │           │           │           │           │
├───────────┼───────────┼───────────┼───────────┼───────────┤
│ 1 = 0001  │           │           │           │           │
├─────

Next, we can create an oracle that specifies outcomes with all zero digits in the value register.
We can use the function below to create such an oracle:

In [47]:
def oracle_match_0_multi(bits, tag_bits):
    q = QuantumRegister(bits)
    qc = QuantumCircuit(q)

    for t in tag_bits:
        qc.x(q[t])

    qc.mcp(pi, [q[t] for t in tag_bits[:-1]], q[len(q) - 1])

    for t in tag_bits:
        qc.x(q[t])

    return qc

We can create the oracle by passing each of the value register qubits as the `tag_bits`:

In [48]:
prepare = build_polynomial_circuit(n_key, n_value, terms)

oracle = oracle_match_0_multi(
    n_key + n_value,
    [n_key + i for i in range(n_value)]
)

qc = grover_circuit(prepare, oracle, 1)

In [49]:
grid_state(qc.run(), n_key, True, True)



╒═══════════╤══════════╤══════════╤═══════════════╤══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10        │ 3 = 11   │
╞═══════════╪══════════╪══════════╪═══════════════╪══════════╡
│ 7 = 0111  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 6 = 0110  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 5 = 0101  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 4 = 0100  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 3 = 0011  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 2 = 0010  │          │          │               │          │
├───────────┼──────────┼──────────┼───────────────┼──────────┤
│ 1 = 0001  │          │          │               │  